In [1]:
from AgentSimulate import Agent
from game.PokerSimple import PokerSimple
import random
import numpy as np
import copy
import tensorflow_probability as tfp
import tensorflow as tf

2022-06-13 13:07:28.700221: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-13 13:07:28.700264: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import random
class MCST:
    def __init__(self, gamestate, agent, policy_networks, perspective, n_runs = 10, n_sim = 10,store_samples = False, training= False):

        self.game = copy.deepcopy(gamestate)
        if self.game.street == 0:
            deck = self.game.deck[1:]
            random.shuffle(deck)
            self.game.deck[1:]=deck
            self.game.hole_1=self.game.deck[1]

        else:
            board = self.game.board
            deck = []
            deck.append(self.game.deck[1])
            deck.extend(self.game.deck[3:])
            random.shuffle(deck)
            self.game.deck[1]=deck[0]
            self.game.deck[3:]= deck[1:]
            self.game.hole_1=self.game.deck[1]

        self.game.run_ranges = True
        self.agent = agent
        self.perspective = perspective
        self.n_runs = n_runs
        self.n_sim = n_sim
        self.store_samples = store_samples
        self.training = training
        self.policy_networks = policy_networks
    def search(self):
        game = self.game
        agent = self.agent
        actions_taken = {0:0, 1:0, 2:0} #to count number of actions taken in simulations
        evs = {} #save evs for each action

        #find legal actions in state
        if game.stacks[0] == 4.5 or game.stacks[1]==4.5: #beginning of round
            legal_actions = [0,1,2]
        elif game.stacks[0] != game.stacks[1]:
            legal_actions = [0,1]
        else:
            legal_actions = [1,2]

        #initialize policy
        game.create_observation(game.next_to_act[0])
        observation = game.observations[game.next_to_act[0]][:]
        if self.store_samples:
            self.observation = observation[:]
        if observation[4] == observation[6]:
            observation.append(1)
        else:
            observation.append(0)

        state = tf.convert_to_tensor([observation])
        #policy = np.array(agent.actor(state))[0]
        policy = np.array([0.33,0.33,0.34])

        """for i in range(self.n_runs):
            action_probabilities = tfp.distributions.Categorical(probs=policy)
            action = int(np.array(action_probabilities.sample()))
            while action not in legal_actions:
                action = int(np.array(action_probabilities.sample()))"""

        for leg_act in legal_actions:
            action = leg_act
            actions_taken[action] += 1
            if action not in evs.keys():
                evs[action] = 0
                saw_flop = agent.saw_flop[:]
                ranges = copy.deepcopy(agent.ranges_vil)
                reward_sim = 0
                next_observations = {}
                for j in range(self.n_sim):
                    agent.ranges_vil = copy.deepcopy(ranges)
                    game_sim = copy.deepcopy(game)
                    #reshuffle deck.
                    if game_sim.street == 0:
                        deck = game_sim.deck[1:]
                        random.shuffle(deck)
                        game_sim.deck[1:]=deck
                        game_sim.hole_1= game_sim.deck[1]
                    else:
                        deck = [game_sim.deck[1]]
                        deck.extend(game_sim.deck[3:])
                        random.shuffle(deck)
                        game_sim.deck[1]=deck[0]
                        game_sim.deck[3:]= deck[1:]
                        game_sim.hole_1= game_sim.deck[1]
                    #add ranges villain
                    game_sim.ranges[1-self.perspective] = agent.ranges_vil[self.perspective]

                    game_sim.implement_action(game_sim.next_to_act[0],action)
                    if (not agent.saw_flop[1-self.perspective]) and game_sim.street ==1:
                        hole_hero = game_sim.hole_0 if self.perspective == 0 else game_sim.hole_1
                        agent.update_range_flop(self.perspective, hole_hero, game_sim.board)
                        game_sim.ranges[1-self.perspective] = agent.ranges_vil[self.perspective]

                    if game_sim.done:
                        reward = game_sim.stacks[self.perspective] -4.5 if game_sim.observations[self.perspective][0] == 0 else game_sim.stacks[self.perspective] - 4
                    else:
                        while (not game_sim.done) and game_sim.next_to_act[0]!=self.perspective: #villain acts
                            if (not agent.saw_flop[1-self.perspective]) and game_sim.street ==1:
                                hole_hero = game_sim.hole_0 if self.perspective == 0 else game_sim.hole_1
                                agent.update_range_flop(self.perspective, hole_hero, game_sim.board)
                                game_sim.ranges[1-self.perspective] = agent.ranges_vil[self.perspective]
                            game_sim.create_observation(1-self.perspective)
                            observation = game_sim.observations[1-self.perspective][:]
                            observation[4] =0
                            #print("obs vil", observation)
                            observation.append(-1)
                            states = np.array([observation for obs in range(5)])
                            states[:,4] = [card for card in range(1,6)]
                            states[:,-1] = (states[:,4]==states[:,6]).astype(int)
                            policy_villain = np.array(agent.actor(states))
                            #print("range_vil", agent.ranges_vil[self.perspective])
                            #print("policy vil", policy_villain)
                            new_policy_vil = (np.expand_dims(agent.ranges_vil[self.perspective],axis=1)*policy_villain).sum(axis=0)
                            #print("new policy vil", new_policy_vil)
                            action_vil = random.choices([0,1,2], weights = new_policy_vil)[0]
                            #print("action_vil", action_vil)
                            agent.update_range_action(self.perspective, observation[:-1], action_vil)
                            game_sim.ranges[1-self.perspective] = agent.ranges_vil[self.perspective]
                            game_sim.implement_action(1-self.perspective,action_vil)
                    if game_sim.done:
                        position_hero = game_sim.position_0 if self.perspective == 0 else game_sim.position_1
                        reward = game_sim.stacks[self.perspective] -4.5 if position_hero == 0 else game_sim.stacks[self.perspective] - 4

                    else:
                        game_sim.create_observation(self.perspective)
                        if tuple(game_sim.observations[self.perspective]) in next_observations.keys():
                            evs_tree = next_observations[tuple(game_sim.observations[self.perspective])]
                        else:
                            new_tree = MCST(game_sim,  agent, self.perspective,self.n_runs,self.n_sim,store_samples=False)
                            evs_tree=new_tree.search()[0]
                            if self.training:
                                next_observations[tuple(game_sim.observations[self.perspective])] = evs_tree
                                train_action = max(evs_tree, key=evs_tree.get)

                                obs = copy.deepcopy(game_sim.observations[0])
                                obs.append(0) if obs[4] != obs[6] else obs.append(1)
                                train_obs = np.expand_dims(np.array(obs),axis=0)
                                print(train_obs)
                                print("label", train_action)
                                print("predicted",self.agent.actor.predict(train_obs))
                                self.agent.actor.fit(train_obs,np.array([train_action]))

                        reward = evs_tree[max(evs_tree, key=evs_tree.get)]
                    reward_sim += reward
                    #print("current ev for action", action, reward_sim/(j+1))
                #reset pre-simulation ranges
                agent.ranges_vil=ranges
                agent.saw_flop=saw_flop



                evs[action]+=(reward_sim/self.n_sim)



        if self.store_samples:
            agent.store_sample(self.observation,policy)
        return evs, policy, actions_taken



In [2]:
agent_0 = Agent()
agent_0.load_models()

2022-06-13 13:08:29.039117: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-13 13:08:29.039218: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-13 13:08:29.039258: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cromi-Lenovo-V15-ADA): /proc/driver/nvidia/version does not exist
2022-06-13 13:08:29.041694: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


... loading models ...


In [5]:
network_0 = agent_0.actor
agent_new = Agent()
network_1 = agent_new.actor
game = PokerSimple(0,1)

In [6]:
#first let's check if montecarlo beats the q-table if it knows its policy and by how much
results = []
game = PokerSimple(0,1)
print("lr",agent_0.actor.optimizer.learning_rate)
from keras import backend as K
K.set_value(agent_0.actor.optimizer.learning_rate, 5e-06)
n_games = 10000
total_chips = 0
for ep in range(n_games):
    print(ep)
    game.reset()
    agent_0.set_range_vil(0,game.hole_0)
    agent_0.saw_flop = [False, False]

    while not game.done:
        if game.next_to_act[0]== 0:
            game.create_observation(0)
            game_sim = copy.deepcopy(game)
            if game_sim.street == 0:
                deck = game_sim.deck[1:]
                random.shuffle(deck)
                game_sim.deck[1:]=deck
                game_sim.hole_1=game_sim.deck[1]

            else:
                board = game_sim.board
                deck = []
                deck.append(game_sim.deck[1])
                deck.extend(game_sim.deck[3:])
                random.shuffle(deck)
                game_sim.deck[1]=deck[0]
                game_sim.deck[3:]= deck[1:]
                game_sim.hole_1=game_sim.deck[1]

            tree = MCST(game_sim,agent_0, 0, n_runs = 10, n_sim = 30, training=True)
            evs, policy, _ = tree.search()
            action = max(evs, key=evs.get)

            print(game.observations[0])
            print("action", action)
            obs = copy.deepcopy(game.observations[0])
            obs.append(0) if obs[4] != obs[6] else obs.append(1)
            train_obs = np.expand_dims(np.array(obs),axis=0)
            agent_0.actor.fit(train_obs,np.array([action]))

            game.implement_action(0, action)
            if (not agent_0.saw_flop[1]) and game.street == 1:
                agent_0.update_range_flop(0, game.hole_0, game.board)


        else:
            game.create_observation(1)
            action = agent_0.choose_action(game.observations[1])
            obs_vil = game.observations[1][:]
            obs_vil[4] = 0
            agent_0.update_range_action(0,obs_vil,action)
            if (not agent_0.saw_flop[1]) and game.street == 1:
                agent_0.update_range_flop(0, game.hole_0, game.board)
            game.implement_action(1, action)

    result = game.stacks[0] - 5
    results.append(result)
    total_chips += result
    #avg_chips = total_chips/(ep+0.00000000001)
    print(np.array(results[-350:]).mean())
    if ep%100 == 0:
        agent_0.print_strategy()

lr <tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=5e-06>
0


NameError: name 'MCST' is not defined

In [22]:
agent_0.save_models()
agent_0.print_strategy()

... saving models ...
--------------------------------------
preflop

sb
 first in

holecard 1 probs [0.392, 0.605, 0.004]
holecard 2 probs [0.065, 0.924, 0.011]
holecard 3 probs [0.007, 0.972, 0.02]
holecard 4 probs [0.001, 0.972, 0.028]
holecard 5 probs [0.0, 0.979, 0.021]
preflop

sb
 complete - push

holecard 1 probs [0.999, 0.001, 0.0]
holecard 2 probs [0.991, 0.009, 0.0]
holecard 3 probs [0.414, 0.586, 0.0]
holecard 4 probs [0.008, 0.992, 0.001]
holecard 5 probs [0.001, 0.998, 0.001]

bb
  to complete

holecard 1 probs [0.024, 0.967, 0.008]
holecard 2 probs [0.003, 0.98, 0.017]
holecard 3 probs [0.0, 0.981, 0.019]
holecard 4 probs [0.0, 0.985, 0.015]
holecard 5 probs [0.0, 0.97, 0.03]

bb
  to push

holecard 1 probs [1.0, 0.0, 0.0]
holecard 2 probs [0.999, 0.001, 0.0]
holecard 3 probs [0.93, 0.07, 0.0]
holecard 4 probs [0.09, 0.91, 0.0]
holecard 5 probs [0.002, 0.998, 0.0]
--------------------------------------
postflop

sb
  to check

board 1
holecard 1 probs [0.0, 0.052, 0.948]

In [7]:
games = []
observations = []
import copy


for card in range(1,6):
    done, player, obs = game.reset()
    while player!= 0 or obs[4] != card:
        done, player, obs = game.reset()
    observations.append(obs)

for card in range(1,6):
    done, player, obs = game.reset()
    while player != 0 or game.hole_0 != card:
        done, player, obs = game.reset()

    game.implement_action(0,1)
    game.implement_action(1,2)
    game.create_observation(0)
    observations.append(game.observations[0][:])

for card in range(1,6):
    done, player, obs = game.reset()
    while player!= 1 or game.hole_0 != card:
        done, player, obs = game.reset()
    game.implement_action(1,1)
    game.create_observation(0)
    observations.append(game.observations[0][:])


for card in range(1,6):
    done, player, obs = game.reset()
    while player!= 1 or game.hole_0 != card:
        done, player, obs = game.reset()

    game.implement_action(1,2)
    game.create_observation(0)
    observations.append(game.observations[0][:])

#postflop - sb to check

for card in range(1,6):
    for board in range(1,6):
        done, player, obs = game.reset()
        while game.hole_0 != card or game.deck[2]!=board or player ==1:
            done, player, obs = game.reset()

        game.implement_action(0,1)
        game.implement_action(1,1)
        game.implement_action(1,1)
        game.create_observation(0)
        observations.append(game.observations[0][:])

#postflop sb to push
for card in range(1,6):
    for board in range(1,6):
        done, player, obs = game.reset()
        while game.hole_0 != card or game.deck[2]!=board or player ==1:
            done, player, obs = game.reset()

        game.implement_action(0,1)
        game.implement_action(1,1)
        game.implement_action(1,2)
        game.create_observation(0)
        observations.append(game.observations[0][:])

#postflop bb first in
for card in range(1,6):
    for board in range(1,6):
        done, player, obs = game.reset()
        while game.hole_0 != card or game.deck[2]!=board or player !=1:
            done, player, obs = game.reset()

        game.implement_action(1,1)
        game.implement_action(0,1)
        game.create_observation(0)
        observations.append(game.observations[0][:])

#postflop bb to push
for card in range(1,6):
    for board in range(1,6):
        done, player, obs = game.reset()
        while game.hole_0 != card or game.deck[2]!=board or player !=1:
            done, player, obs = game.reset()

        game.implement_action(1,1)
        game.implement_action(0,1)
        game.implement_action(0,1)
        game.implement_action(1,2)
        game.create_observation(0)
        observations.append(game.observations[0][:])

In [11]:
X_lst = []
for obs in observations:
    new_obs = obs[:]
    if obs[4] == obs[6]:
        new_obs.append(1)
    else:
        new_obs.append(0)
    X_lst.append(new_obs)
X = np.array(X_lst)

array([[ 0. ,  4.5,  4. , ..., -1. , -1. ,  0. ],
       [ 0. ,  4.5,  4. , ..., -1. , -1. ,  0. ],
       [ 0. ,  4.5,  4. , ..., -1. , -1. ,  0. ],
       ...,
       [ 1. ,  4. ,  0. , ..., -1. , -1. ,  0. ],
       [ 1. ,  4. ,  0. , ..., -1. , -1. ,  0. ],
       [ 1. ,  4. ,  0. , ..., -1. , -1. ,  1. ]])

In [29]:
tf.math.reduce_mean([agent_0.actor(X), agent_new.actor(X)], axis =0 )

<tf.Tensor: shape=(120, 3), dtype=float32, numpy=
array([[0.3813026 , 0.45830756, 0.16038986],
       [0.21381715, 0.61966574, 0.16651708],
       [0.1806605 , 0.6452952 , 0.17404431],
       [0.17182036, 0.6478596 , 0.18032005],
       [0.1695121 , 0.6535861 , 0.1769018 ],
       [0.6650555 , 0.16434655, 0.17059794],
       [0.6569918 , 0.17325842, 0.16974984],
       [0.36676505, 0.46360618, 0.16962877],
       [0.16344234, 0.66632366, 0.17023404],
       [0.15992208, 0.6696575 , 0.17042041],
       [0.1870022 , 0.6405635 , 0.17243433],
       [0.17447431, 0.649184  , 0.17634176],
       [0.16993655, 0.65337884, 0.17668454],
       [0.16673993, 0.6590132 , 0.1742469 ],
       [0.1660676 , 0.6521627 , 0.18176974],
       [0.66315067, 0.16699395, 0.16985534],
       [0.66209126, 0.16746858, 0.17044017],
       [0.61383945, 0.21590923, 0.17025135],
       [0.19545035, 0.63373876, 0.17081085],
       [0.16132897, 0.66801363, 0.17065734],
       [0.17579909, 0.18119788, 0.643003  ],
     

In [32]:
tf.concat([agent_0.actor, agent_new.actor], axis=0)

ValueError: Attempt to convert a value (<keras.engine.sequential.Sequential object at 0x7fe284362eb0>) with an unsupported type (<class 'keras.engine.sequential.Sequential'>) to a Tensor.

In [25]:
agent_new.actor(X)

<tf.Tensor: shape=(120, 3), dtype=float32, numpy=
array([[0.370887  , 0.3119702 , 0.31714278],
       [0.36257488, 0.31519362, 0.32223144],
       [0.35400337, 0.3182405 , 0.3277562 ],
       [0.34291956, 0.32400143, 0.33307895],
       [0.33894023, 0.32823357, 0.33282614],
       [0.3313187 , 0.3274854 , 0.34119585],
       [0.32299075, 0.33750993, 0.33949935],
       [0.3192271 , 0.3415809 , 0.33919203],
       [0.31896687, 0.3411063 , 0.3399269 ],
       [0.31927618, 0.34105095, 0.3396729 ],
       [0.3496799 , 0.31379184, 0.33652827],
       [0.34567803, 0.31873983, 0.33558214],
       [0.33944026, 0.32610065, 0.33445907],
       [0.33341804, 0.33287612, 0.33370584],
       [0.33212528, 0.33452863, 0.33334613],
       [0.32639387, 0.33389542, 0.33971068],
       [0.32491037, 0.3342093 , 0.34088033],
       [0.32349452, 0.3360031 , 0.34050235],
       [0.3225688 , 0.3358194 , 0.34161174],
       [0.32138342, 0.3373414 , 0.34127522],
       [0.35159364, 0.31018436, 0.338222  ],
     

In [26]:
agent_0.actor(X)

<tf.Tensor: shape=(120, 3), dtype=float32, numpy=
array([[3.91718179e-01, 6.04644895e-01, 3.63693992e-03],
       [6.50594383e-02, 9.24137831e-01, 1.08027197e-02],
       [7.31763151e-03, 9.72349942e-01, 2.03324147e-02],
       [7.21152988e-04, 9.71717656e-01, 2.75611449e-02],
       [8.39458808e-05, 9.78938580e-01, 2.09774580e-02],
       [9.98792291e-01, 1.20767159e-03, 2.20092993e-08],
       [9.90992785e-01, 9.00692772e-03, 3.26482876e-07],
       [4.14303005e-01, 5.85631490e-01, 6.55118420e-05],
       [7.91781675e-03, 9.91541088e-01, 5.41180547e-04],
       [5.67982264e-04, 9.98264134e-01, 1.16792344e-03],
       [2.43244953e-02, 9.67335105e-01, 8.34037643e-03],
       [3.27060348e-03, 9.79628086e-01, 1.71013735e-02],
       [4.32853296e-04, 9.80657041e-01, 1.89100280e-02],
       [6.18242193e-05, 9.85150278e-01, 1.47879822e-02],
       [9.93204230e-06, 9.69796717e-01, 3.01933642e-02],
       [9.99907494e-01, 9.24674969e-05, 5.71738906e-11],
       [9.99272168e-01, 7.27860723e-04

In [31]:
ys = []
for obs in observations:
    policy = [0,0,0]
    action = choose_action_q(obs)
    policy[action] = 1
    ys.append(policy)
ys = np.array(ys)
ys

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1,

In [28]:
X = copy.deepcopy(observations)
for obs in X:
    if obs[4] == obs[6]:
        obs.append(1)
    else:
        obs.append(0)
X = np.array(X)
X

array([[ 0. ,  4.5,  4. , ..., -1. , -1. ,  0. ],
       [ 0. ,  4.5,  4. , ..., -1. , -1. ,  0. ],
       [ 0. ,  4.5,  4. , ..., -1. , -1. ,  0. ],
       ...,
       [ 1. ,  4. ,  0. , ..., -1. , -1. ,  0. ],
       [ 1. ,  4. ,  0. , ..., -1. , -1. ,  0. ],
       [ 1. ,  4. ,  0. , ..., -1. , -1. ,  1. ]])

In [78]:
from keras import backend as K
K.set_value(agent_0.actor.optimizer.learning_rate, 1e-04)
agent_0.actor.fit(X,ys, epochs = 1000, batch_size=1)

NameError: name 'ys' is not defined

In [20]:
agent_random = agent_0
n_games = 10000
total_chips = 0
for ep in range(n_games):
    print(ep)
    game.reset()
    agent_random.set_range_vil(0,game.hole_0)
    agent_random.saw_flop = [False, False]

    while not game.done:
        if game.next_to_act[0]== 0:
            game.create_observation(0)
            game_sim = copy.deepcopy(game)
            if game_sim.street == 0:
                deck = game_sim.deck[1:]
                random.shuffle(deck)
                game_sim.deck[1:]=deck
                game_sim.hole_1=game_sim.deck[1]

            else:
                board = game_sim.board
                deck = []
                deck.append(game_sim.deck[1])
                deck.extend(game_sim.deck[3:])
                random.shuffle(deck)
                game_sim.deck[1]=deck[0]
                game_sim.deck[3:]= deck[1:]
                game_sim.hole_1=game_sim.deck[1]


            tree = MCST(game_sim,agent_random, 0, n_runs = 10, n_sim = 20)
            evs, policy, _ = tree.search()
            action = max(evs, key=evs.get)

            game.implement_action(0, action)
            if (not agent_random.saw_flop[1]) and game.street == 1:
                agent_random.update_range_flop(0, game.hole_0, game.board)


        else:
            game.create_observation(1)
            action = choose_action_q(game.observations[1])
            obs_vil = game.observations[1][:]
            obs_vil[4] = 0
            agent_random.update_range_action(0,obs_vil,action)
            if (not agent_random.saw_flop[1]) and game.street == 1:
                agent_random.update_range_flop(0, game.hole_0, game.board)
            game.implement_action(1, action)

    result = game.stacks[0] - 5
    total_chips += result
    avg_chips = total_chips/(ep+0.00000000001)
    print("total", total_chips, "avg", avg_chips)

0
total 1.0 avg 100000000000.0
1
total 2.0 avg 1.99999999998
2
total 1.0 avg 0.4999999999975
3
total 0.0 avg 0.0
4
total 1.0 avg 0.249999999999375
5
total 2.0 avg 0.3999999999992
6
total 1.5 avg 0.24999999999958333
7
total 2.5 avg 0.35714285714234695
8
total 3.5 avg 0.43749999999945316
9
total 4.5 avg 0.4999999999994445
10
total 4.5 avg 0.44999999999955004
11
total 5.5 avg 0.4999999999995455
12
total 6.5 avg 0.5416666666662153
13
total 7.5 avg 0.5769230769226332
14
total 7.5 avg 0.5357142857139031
15
total 6.5 avg 0.43333333333304447
16
total 6.5 avg 0.4062499999997461
17
total 1.5 avg 0.08823529411759515
18
total 1.0 avg 0.05555555555552469
19
total 2.0 avg 0.10526315789468144
20
total 3.0 avg 0.149999999999925
21
total -2.0 avg -0.09523809523804988
22
total -3.0 avg -0.13636363636357438
23
total -3.0 avg -0.13043478260863894
24
total -2.0 avg -0.0833333333332986
25
total -1.0 avg -0.039999999999984
26
total 0.0 avg 0.0
27
total -1.0 avg -0.03703703703702332
28
total 0.0 avg 0.0
29
to

KeyboardInterrupt: 

In [98]:
from sklearn.model_selection import train_test_split
X0= agent_0.memory.state_memory[:agent_0.memory.mem_cntr]
y0 = agent_0.memory.policy_memory[:agent_0.memory.mem_cntr]
#X1= agent_1.memory.state_memory[:agent_1.memory.mem_cntr]
#y1 = agent_1.memory.policy_memory[:agent_1.memory.mem_cntr]
#X=np.concatenate((X0,X1))
#y=np.concatenate((y0,y1))
X_train, X_test, y_train, y_test = train_test_split(X0, y0, test_size=0.10, random_state=42)

In [34]:
from keras import backend as K
K.set_value(agent_0.actor.optimizer.learning_rate, 1e-03)
agent_0.actor.fit(X_train,y_train, batch_size=4, epochs=5000)

NameError: name 'X_train' is not defined

In [91]:
X

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.0,4.0,0.0,0.0,1.0,6.0,0.0,0.5,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
1,0.0,4.0,0.0,0.0,2.0,6.0,0.0,0.5,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
2,0.0,4.0,0.0,0.0,3.0,6.0,0.0,0.5,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
3,0.0,4.0,0.0,0.0,4.0,6.0,0.0,0.5,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
4,0.0,4.0,0.0,0.0,5.0,6.0,0.0,0.5,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,1.0,4.0,4.0,1.0,5.0,2.0,1.0,0.5,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
116,1.0,4.0,4.0,1.0,5.0,2.0,2.0,0.5,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
117,1.0,4.0,4.0,1.0,5.0,2.0,3.0,0.5,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0
118,1.0,4.0,4.0,1.0,5.0,2.0,4.0,0.5,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0


In [100]:
from keras import backend as K
K.set_value(agent_0.actor.optimizer.learning_rate, 0.00001)
agent_0.actor.fit(X,y, epochs=10000, batch_size=4)

Epoch 1/10000
30/30 [==============================] - 0s 2ms/step - loss: 0.0799
Epoch 2/10000
30/30 [==============================] - 0s 2ms/step - loss: 0.0766
Epoch 3/10000
30/30 [==============================] - 0s 2ms/step - loss: 0.0739
Epoch 4/10000
30/30 [==============================] - 0s 2ms/step - loss: 0.0717
Epoch 5/10000
30/30 [==============================] - 0s 2ms/step - loss: 0.0700
Epoch 6/10000
30/30 [==============================] - 0s 2ms/step - loss: 0.0685
Epoch 7/10000
30/30 [==============================] - 0s 2ms/step - loss: 0.0671
Epoch 8/10000
30/30 [==============================] - 0s 2ms/step - loss: 0.0658
Epoch 9/10000
30/30 [==============================] - 0s 2ms/step - loss: 0.0647
Epoch 10/10000
30/30 [==============================] - 0s 2ms/step - loss: 0.0638
Epoch 11/10000
30/30 [==============================] - 0s 2ms/step - loss: 0.0628
Epoch 12/10000
30/30 [==============================] - 0s 2ms/step - loss: 0.0619
Epoch 13/1000

In [94]:
agent_0.print_strategy()

--------------------------------------
preflop

sb
 first in

holecard 1 probs [0.385, 0.393, 0.222]
holecard 2 probs [0.256, 0.444, 0.3]
holecard 3 probs [0.016, 0.375, 0.609]
holecard 4 probs [0.006, 0.732, 0.262]
holecard 5 probs [0.001, 0.895, 0.104]
preflop

sb
 complete - push

holecard 1 probs [0.913, 0.083, 0.004]
holecard 2 probs [0.84, 0.156, 0.004]
holecard 3 probs [0.367, 0.631, 0.001]
holecard 4 probs [0.094, 0.905, 0.0]
holecard 5 probs [0.011, 0.989, 0.0]

bb
  to complete

holecard 1 probs [0.002, 0.936, 0.063]
holecard 2 probs [0.001, 0.836, 0.163]
holecard 3 probs [0.001, 0.743, 0.256]
holecard 4 probs [0.0, 0.527, 0.472]
holecard 5 probs [0.0, 0.685, 0.315]

bb
  to push

holecard 1 probs [0.961, 0.037, 0.002]
holecard 2 probs [0.924, 0.075, 0.002]
holecard 3 probs [0.16, 0.84, 0.0]
holecard 4 probs [0.003, 0.996, 0.001]
holecard 5 probs [0.0, 0.999, 0.0]
--------------------------------------
postflop

sb
  to check

board 1
holecard 1 probs [0.0, 0.064, 0.936]
hole

In [ ]:
agent_0.actor.evaluate(X_test, y_test)

In [7]:
X = agent_0.memory.state_memory[:agent_0.memory.mem_cntr]
y = agent_0.memory.policy_memory[:agent_0.memory.mem_cntr]

In [28]:
agent_0 = Agent(hidden_layers=(256,128,64,32,16,8,4))
agent_0.load_models()
agent_0.print_strategy()

... loading models ...
--------------------------------------
preflop

sb
 first in

holecard 1 probs [0.004, 0.769, 0.227]
holecard 2 probs [0.001, 0.684, 0.315]
holecard 3 probs [0.0, 0.581, 0.419]
holecard 4 probs [0.0, 0.307, 0.693]
holecard 5 probs [0.0, 0.12, 0.88]
preflop

sb
 complete - push

holecard 1 probs [0.993, 0.007, 0.0]
holecard 2 probs [0.979, 0.021, 0.0]
holecard 3 probs [0.35, 0.648, 0.002]
holecard 4 probs [0.003, 0.994, 0.002]
holecard 5 probs [0.0, 0.997, 0.003]

bb
  to complete

holecard 1 probs [0.003, 0.706, 0.292]
holecard 2 probs [0.001, 0.685, 0.315]
holecard 3 probs [0.0, 0.552, 0.448]
holecard 4 probs [0.0, 0.356, 0.644]
holecard 5 probs [0.0, 0.255, 0.745]

bb
  to push

holecard 1 probs [0.996, 0.004, 0.0]
holecard 2 probs [0.988, 0.012, 0.0]
holecard 3 probs [0.425, 0.575, 0.001]
holecard 4 probs [0.002, 0.998, 0.001]
holecard 5 probs [0.0, 0.999, 0.001]
--------------------------------------
postflop

sb
  to check

board 1
holecard 1 probs [0.0, 0.0

In [6]:
import pickle
import numpy as np

with open('q_simple', 'rb') as q_simple_file:
    q = pickle.load(q_simple_file)
def choose_action_q(observation):

    hole = observation[4]
    #sb:
    if observation[0] == 0:
        if observation[3] == 0: #preflop
            if observation[-10] ==-1: #first in
                action = np.argmax(q['sb'][hole]['preflop']['preflop_fi'])

            else: # bb must have gone allin, if sb has another action to perform
                action = np.argmax(q['sb'][hole]['preflop']['preflop_to_push'])


        else: #postflop
            board = observation[6]
            if observation[2] != 0: #bb did not push
                action = np.argmax(q['sb'][hole]['postflop'][board]['to_check'])
                if action == 0:
                    action =1
                else:
                    action =2
            else:
                action = np.argmax(q['sb'][hole]['postflop'][board]['to_push'])


    #bb
    else:
        if observation[3] == 0: #preflop
            if observation[-10] == 0.5: #sb completed
                action = np.argmax(q['bb'][hole]['preflop']['to_call'])
                if action == 0:
                    action =1
                else:
                    action =2
            else: #sb pushed
                action = np.argmax(q['bb'][hole]['preflop']['to_push'])

        else:
            board = observation[6]
            if observation[2]!= 0: #bb is first in
                action = np.argmax(q['bb'][hole]['postflop'][board]['postflop_fi'])
                if action == 0:
                    action =1
                else:
                    action =2
            else: #bb checked and sb pushed
                action = np.argmax(q['bb'][hole]['postflop'][board]['postflop_to_push'])

    return action

In [ ]:
agent_0.print_strategy()

In [23]:
n_games = 10000
total_chips = 0
for ep in range(n_games):
    print(ep)
    game.reset()

    while not game.done:
        if game.next_to_act[0]==0:
            game.create_observation(0)
            action = agent_0.choose_action(game.observations[0])
            game.implement_action(0,action)
        else:
            game.create_observation(1)
            action = choose_action_q(game.observations[1])
            game.implement_action(1,action)
    result = game.stacks[0] - 5
    total_chips += result
    avg_chips = total_chips/(ep+0.00000000001)
    print("total", total_chips, "avg", avg_chips)


0
total 1.0 avg 100000000000.0
1
total 0.0 avg 0.0
2
total 1.0 avg 0.4999999999975
3
total 2.0 avg 0.6666666666644444
4
total 3.0 avg 0.749999999998125
5
total 2.0 avg 0.3999999999992
6
total 1.0 avg 0.16666666666638888
7
total 2.0 avg 0.2857142857138775
8
total 3.0 avg 0.3749999999995313
9
total 3.0 avg 0.333333333332963
10
total -2.0 avg -0.1999999999998
11
total -1.0 avg -0.09090909090900827
12
total -6.0 avg -0.4999999999995834
13
total -6.0 avg -0.4615384615381065
14
total -7.0 avg -0.4999999999996429
15
total -8.0 avg -0.5333333333329778
16
total -9.0 avg -0.5624999999996484
17
total -9.0 avg -0.5294117647055709
18
total -9.5 avg -0.5277777777774846
19
total -10.5 avg -0.5526315789470775
20
total -9.5 avg -0.4749999999997625
21
total -8.5 avg -0.404761904761712
22
total -9.5 avg -0.43181818181798554
23
total -10.5 avg -0.4565217391302363
24
total -9.5 avg -0.3958333333331684
25
total -14.5 avg -0.579999999999768
26
total -13.5 avg -0.5192307692305695
27
total -12.5 avg -0.4629629

KeyboardInterrupt: 

In [ ]:
game.run_ranges

In [62]:
agent_new.print_strategy()

--------------------------------------
preflop

sb
 first in

holecard 1 probs [0.04, 0.96, 0.0]
holecard 2 probs [0.0, 1.0, 0.0]
holecard 3 probs [0.0, 1.0, 0.0]
holecard 4 probs [0.0, 0.813, 0.187]
holecard 5 probs [0.0, 0.131, 0.869]
preflop

sb
 complete - push

holecard 1 probs [1.0, 0.0, 0.0]
holecard 2 probs [1.0, 0.0, 0.0]
holecard 3 probs [1.0, 0.0, 0.0]
holecard 4 probs [1.0, 0.0, 0.0]
holecard 5 probs [0.0, 1.0, 0.0]

bb
  to complete

holecard 1 probs [0.001, 0.999, 0.0]
holecard 2 probs [0.0, 1.0, 0.0]
holecard 3 probs [0.0, 0.924, 0.076]
holecard 4 probs [0.0, 0.206, 0.794]
holecard 5 probs [0.0, 0.066, 0.934]

bb
  to push

holecard 1 probs [1.0, 0.0, 0.0]
holecard 2 probs [1.0, 0.0, 0.0]
holecard 3 probs [1.0, 0.0, 0.0]
holecard 4 probs [1.0, 0.0, 0.0]
holecard 5 probs [0.0, 1.0, 0.0]
--------------------------------------
postflop

sb
  to check

board 1
holecard 1 probs [0.0, 0.115, 0.885]
holecard 2 probs [0.0, 0.007, 0.993]
holecard 3 probs [0.0, 0.007, 0.993]
holec